#                                      **SIIM Melanoma Competition**

This competition is an interesting and important one from the pov of improvments in medical image diagnosis we can have using modern and state of the art deep learning models and computer vision. Melanoma, as the competyition overview states,is responsible for 75% of skin cancer deaths, despite being the least common skin cancer and the American Cancer Society estimates over 100,000 new melanoma cases will be diagnosed in 2020. Current AI techniques have not been much succesful but the hosts have expected the large pool of experienced and efficient machine learning and data scientists on Kaggle will be able to provide models that have better results and help in early diagnosis.

## **Version 8** --  Resnext50_32x4d model used on 224x224 uncropped image. [LB - 0.87]

## **Version 15** -- Efficientnet b2 model used on uncropped 224x224 image. [LB - 0.892]

## **Version 24** -- Efficientnet b3 model used on square centre cropped 224x224 image posted by Chris Deotte. [LB-- 0.910] 

## **Version 28** -- Changed to using Focal loss as loss function.

Started experimenting with higher image sizes. Training on gpu was very slow and frustrating,had to move to tpus.Given I have never used tpus before either in pytorch or tf, Abhisek Thakur's accelerator-power-hour [workshop](https://www.youtube.com/watch?v=DEuvGh4ZwaY&feature=youtu.be) on kaggle was very helpful for me to change my code to tpu version. Thanks to Chris for making so many important discussions in this competition and sharing the datasets and to Abhisek for his videos on youtube and public notebooks.

## **Version 40** -- Efficientnet b4 on 384x384 centre cropped images by Chris Deotte on TPU. [LB--9205]

**Dataset and discussion links**

224x224 square centre cropped by Chris Deotte - [link](https://www.kaggle.com/cdeotte/jpeg-melanoma-256x256?select=test.csv)

Pytorch or Tensorflow jpegs discussion post by Chris Deotte - [link](https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/164910)

melanoma merged external data 512x512 jpeg by Alex Shonenkov - [link](https://www.kaggle.com/shonenkov/melanoma-merged-external-data-512x512-jpeg)

**Setting up the environment**

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip install wtfml
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=bc4b2f07cb3dc3ea2f6d5d55ec64a0ad24189c615e9b4ec4ff491a5ce95a54be
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
Successfully built efficientnet-pytorch


In [ ]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**Importing packages**

In [ ]:
import warnings
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm 
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import warnings
import gc

warnings.filterwarnings("ignore")

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision

import cv2

import numpy as np 
import pandas as pd
import os

from torch.utils.data import DataLoader,TensorDataset,Dataset
import matplotlib.pyplot as plt
import albumentations
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from efficientnet_pytorch import EfficientNet

from wtfml.utils import EarlyStopping

In [ ]:
train_df = pd.read_csv('/kaggle/input/jpeg-melanoma-384x384/train.csv')  #/kaggle/input/siim-isic-melanoma-classification/train.csv
train_df.head()

making 5 fold divisions on the dataset csv

In [ ]:
df = train_df.sample(frac=1).reset_index(drop=True)
df['kfold'] = -1
y = train_df.target.values
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=True)
idx = kf.get_n_splits(X=df,y=y)
print(idx)
for fold,(x,y) in enumerate(kf.split(X=df,y=y)):
    df.loc[y,'kfold'] = fold

In [22]:
#df.to_csv('train_fold_tpu.csv',index=False)
df = pd.read_csv('/kaggle/input/tpu-csv/train_fold_tpu.csv')
#df = pd.read_csv('/kaggle/input/melanoma-merged-external-data-512x512-jpeg/folds.csv')

In [23]:
df.head(10) 

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,tfrecord,width,height,kfold
0,ISIC_7658533,IP_4865295,male,60.0,torso,nevus,benign,0,3,4288,2848,4
1,ISIC_4521497,IP_0656529,male,60.0,torso,unknown,benign,0,3,6000,4000,2
2,ISIC_4433832,IP_3219832,female,55.0,torso,nevus,benign,0,14,1872,1053,0
3,ISIC_7903213,IP_8329777,female,35.0,torso,nevus,benign,0,7,1872,1053,3
4,ISIC_7831274,IP_7911457,female,50.0,lower extremity,nevus,benign,0,0,1872,1053,0
5,ISIC_5592257,IP_6078411,male,35.0,torso,unknown,benign,0,11,6000,4000,1
6,ISIC_4785487,IP_1355796,male,60.0,upper extremity,nevus,benign,0,9,1872,1053,3
7,ISIC_6547275,IP_1652699,male,55.0,torso,unknown,benign,0,1,5184,3456,4
8,ISIC_4927560,IP_6887429,male,45.0,upper extremity,nevus,benign,0,8,1872,1053,1
9,ISIC_8922111,IP_7804786,male,50.0,upper extremity,unknown,benign,0,4,5184,3456,1


# **Define custom dataset**

define helper function for image augmentation using albumentations library

inp : image path,image name,valid as a boolean showing whether the image is from train or validation set

out : a image vector of type torch.tensor and shape (3,256,256)

Also training images are augmented in different ways whereas validation images should keep their actual identity and hence they are only normalized.

In [6]:
class CustomDataset(Dataset):
    def __init__(self,path,name,target,aug):
        super(CustomDataset,self).__init__()
        self.path = path
        self.name = name
        self.target = target
        self.aug = aug
        
        
    def __len__(self):
        return len(self.name)
    
    def __getitem__(self,index):
        
        im_name = self.name[index]
        y = self.target[index]
        img_path = os.path.join(self.path,im_name + '.jpg')
        img = cv2.resize(cv2.imread(img_path),dsize=(384,384))
        image = self.aug(image=img)
        l = image['image']
        image = np.transpose(l, (2, 0, 1)).astype(np.float32)
        
        return torch.tensor(image,dtype=torch.float),torch.tensor(y)
        
        

# **Define dataloader for tpu**

In [ ]:
class Data_Loader():
    def __init__(self,path,name,target,aug):
        self.path = path
        self.name = name
        self.target = target
        self.aug = aug
        self.dataset = CustomDataset(self.path,self.name,self.target,self.aug)
        
    def get(self,batch_size,shuffle,num_workers):
        
        sampler = torch.utils.data.distributed.DistributedSampler(self.dataset,
                                                                  num_replicas = xm.xrt_world_size(),
                                                                  rank = xm.get_ordinal(),
                                                                  shuffle = shuffle)
        dataloader = torch.utils.data.DataLoader(self.dataset,
                                                 batch_size=batch_size,
                                                 shuffle=False,
                                                 sampler=sampler,
                                                 num_workers=num_workers)
        return dataloader
        
        

# **Set up network architecture**

In [4]:
class EffNet(nn.Module):
    def __init__(self,model='b4'):
        super(EffNet,self).__init__()
        
        model_name = 'efficientnet' + model
        self.feature = EfficientNet.from_pretrained("efficientnet-b4")
        self.drop = nn.Dropout(0.3)
        self.l0 = nn.Linear(1792,1) # b3 - 1536 b2 - 1408
        
        
    def forward(self,img):
        batch_size = img.shape[0]
        
        x = self.feature.extract_features(img)
        #print(x.shape)
        
        x = nn.functional.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #print(x.shape)
        
        x = self.drop(x)
        #print(x.shape)
        out = self.l0(x)
        #print(out.shape)
        
        return out

# **Focal Loss**

Changed to focal loss which is a modified loss function based on cross entropy loss specially used in cases where there is high imbalannce in the nature of datasets.

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self,alpha=1,gamma=2):
        super(FocalLoss,self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self,preds,truth):
        criterion = nn.BCEWithLogitsLoss()
        logits = criterion(preds,truth.unsqueeze(-1).type_as(preds))
        pt = torch.exp(-logits)
        focal_loss = self.alpha*(1-pt)**self.gamma*logits
        
        return torch.mean(focal_loss)

# **Training**

In [ ]:
def train(model,fold):
    
    batch_t = 128
    batch_v = 128
    best_score = 0
    device = xm.xla_device() 
    
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_transpose = albumentations.Compose([
                albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
                albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
                albumentations.Flip(p=0.5)
                #albumentations.CenterCrop(150,150,always_apply=True)
            ])
    valid_transpose = albumentations.Compose(
        [ albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True) 
        ])
    
    image_path = '/kaggle/input/jpeg-melanoma-384x384/train/' #'/kaggle/input/siic-isic-224x224-images/train/'
    train_df = df[df.kfold!=fold].reset_index(drop=True)  #kfold to fold
    valid_df = df[df.kfold==fold].reset_index(drop=True)
    train_im = train_df.image_name.values.tolist()
    train_y = train_df.target.values
    valid_im = valid_df.image_name.values.tolist()
    valid_y = valid_df.target.values
    train_dataset = Data_Loader(image_path,train_im,train_y,
                                train_transpose).get(batch_size=batch_t,shuffle=True,num_workers=4)
    valid_dataset = Data_Loader(image_path,valid_im,valid_y,
                               valid_transpose).get(batch_size=batch_v,shuffle=False,num_workers=4)
    
    
    
    
    
    #model = Resnext50_32x4d()
    #model = EffNet()
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(    #to update the learning rate if model auc score does not increase
        optimizer,                                             #for 3 succesive epochs
        patience=3,           
        threshold=0.001,
        mode="max"
    )

    es = EarlyStopping(patience=5, mode="max",tpu=True)  #early stopping function to stop training if auc score does not increase over 5 epochs
    criterion = nn.BCEWithLogitsLoss()
    #criterion = FocalLoss()
    epochs = 15
    best_score = 0
    
    
    
    for epoch in range(epochs):
            #train mode for training the model and updating the losses
            model.train()
            batch = 0
            #para_loader = pl.ParallelLoader(train_dataset,[device])
            #train_loader = para_loader.per_device_loader(device)
        
            #for _,(train_data,label) in enumerate(train_loader):
            for train_data,label in train_dataset:
                train_data = train_data.to(device)
                label = torch.tensor(label,dtype = torch.float32)
                label = label.to(device)
                
                optimizer.zero_grad()
                out = model(train_data)
                loss = criterion(out,label.unsqueeze(1).type_as(out))
                #loss = criterion(out,label)
                batch +=1
                del train_data,label
                gc.collect()
                if batch%100==0 : print("EPOCH {}  Loss {}  batch  {}".format(epoch,loss.item(),batch))
                
                loss.backward()
                xm.optimizer_step(optimizer,barrier=True)
            #evaluate mode to evaluate the model on cv and update learning rate based on auc score
            #del para_loader,train_loader
            gc.collect()
            model.eval()
            preds = []
            batch = 0
            #para_loader = pl.ParallelLoader(valid_dataset,[device])
            #valid_loader = para_loader.per_device_loader(device)
            
            #for _,(valid_data,valid_label) in enumerate(valid_dataset):
            for valid_data,valid_label in valid_dataset:
                valid_data = valid_data.to(device)
                valid_label = torch.tensor(valid_label,dtype = torch.float32)
                valid_label = valid_label.to(device)
                batch +=1
                
                
                with torch.no_grad():
                    out = model(valid_data)
                    #loss = criterion(out,valid_label)
                    loss = criterion(out,valid_label.unsqueeze(1).type_as(out))
                    preds.append(out.cpu())
                    if batch%50==0 : xm.master_print('Valid Loss {}  batch  {}'.format(loss.item(),batch))
                del valid_data,valid_label
                gc.collect()
            #del para_loader,valid_loader
            gc.collect()
            pred=np.vstack((preds)).ravel()
            #print('pred',pred)
            auc_score = roc_auc_score(valid_y.astype(np.float32),pred)
            print("EPOCH {}  AUC Score {}".format(epoch,auc_score))
            schedular.step(auc_score)
            es(auc_score, model, model_path=f"model_fold_{fold}.bin")
            if es.early_stop:
                print("Early stopping")
                break
            gc.collect()

In [ ]:
model = EffNet()

In [ ]:
def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    a = train(model,0)

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

In [ ]:
train(model,0)
train(model,1)
train(model,2)
train(model,3)
train(model,4)

gpu code that was used previously

In [3]:
def image_aug(path,image_name,valid=False):
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_transpose = albumentations.Compose([
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
            albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
            albumentations.Flip(p=0.5)
            #albumentations.CenterCrop(150,150,always_apply=True)
        ])
    valid_transpose = albumentations.Compose(
    [ albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True) 
    ])
    
    if valid==True :
        
            im_path = os.path.join(path,image_name + '.jpg')
            #img = cv2.imread(im_path)
            img = cv2.resize(cv2.imread(im_path),dsize=(384,384))
            aug = valid_transpose(image=img)
            l = aug['image']
            #print("Validation set image augmented")
               
    else:
        
            im_path = os.path.join(path,image_name + '.jpg')
            img = cv2.resize(cv2.imread(im_path),dsize=(384,384))
            aug = train_transpose(image=img)
            l =aug['image']
            #print("Train set image augmented")
            
    image = np.transpose(l, (2, 0, 1)).astype(np.float32)
    return torch.tensor(image, dtype=torch.float)



class Data_Loader(Dataset):
    def __init__(self,image_path,im_name,target,valid=False):
        self.name = im_name
        self.target = target
        self.path = image_path
        self.valid = valid
        
    def __len__(self):
        return (len(self.name))
    
    def __getitem__(self,index):
        
        if self.valid==False:
            im = self.name[index]
            self.train_y = self.target[index]
            im_tensor = image_aug(self.path,im)
            
            return im_tensor,self.train_y
        
        else:
            im = self.name[index]
            self.valid_y = self.target[index]
            im_tensor = image_aug(self.path,im,valid=True)
            
            return im_tensor,self.valid_y
        
        
        
        
        
def train(fold):
    
    batch_t = 16
    batch_v = 16
    best_score = 0
    device = 'cuda'
    image_path = '/kaggle/input/jpeg-melanoma-384x384/train/' #'/kaggle/input/siic-isic-224x224-images/train/'
    train_df = df[df.kfold!=fold].reset_index(drop=True)  #kfold to fold
    valid_df = df[df.kfold==fold].reset_index(drop=True)
    train_im = train_df.image_name.values.tolist()
    train_y = train_df.target.values
    valid_im = valid_df.image_name.values.tolist()
    valid_y = valid_df.target.values
    train_dataset = Data_Loader(image_path,train_im,train_y)
    train_dataset = DataLoader(train_dataset,batch_t,shuffle=False,num_workers=4)
    valid_dataset = Data_Loader(image_path,valid_im,valid_y,valid=True)
    valid_dataset = DataLoader(valid_dataset,batch_v,shuffle=False,num_workers=4)
    
    
    
    
    
    #model = Resnext50_32x4d()
    model = EffNet()
    model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(    #to update the learning rate if model auc score does not increase
        optimizer,                                             #for 3 succesive epochs
        patience=3,           
        threshold=0.001,
        mode="max"
    )

    es = EarlyStopping(patience=5, mode="max")  #early stopping function to stop training if auc score does not increase over 5 epochs
    criterion = nn.BCEWithLogitsLoss()
    #criterion = FocalLoss()
    epochs = 25
    best_score = 0
    
    
    
    for epoch in range(epochs):
            #train mode for training the model and updating the losses
            model.train()
            batch = 0
        
            for train_data,label in train_dataset:
                train_data = train_data.to(device)
                label = torch.tensor(label,dtype = torch.float32)
                label = label.to(device)
                
                optimizer.zero_grad()
                out = model(train_data)
                loss = criterion(out,label.unsqueeze(1).type_as(out))
                #loss = criterion(out,label)
                batch +=1
                if batch%200==0 : print("EPOCH {}  Loss {}  batch  {}".format(epoch,loss.item(),batch))
                
                loss.backward()
                optimizer.step()
            #evaluate mode to evaluate the model on cv and update learning rate based on auc score
            model.eval()
            preds = []
            batch = 0
            for valid_data,valid_label in valid_dataset:
                valid_data = valid_data.to(device)
                valid_label = torch.tensor(valid_label,dtype = torch.float32)
                valid_label = valid_label.to(device)
                batch +=1
                
                
                with torch.no_grad():
                    out = model(valid_data)
                    #loss = criterion(out,valid_label)
                    loss = criterion(out,valid_label.unsqueeze(1).type_as(out))
                    preds.append(out.cpu())
                    if batch%50==0 : print('Valid Loss {}  batch  {}'.format(loss.item(),batch))

            pred=np.vstack((preds)).ravel()
            #print('pred',pred)
            auc_score = roc_auc_score(valid_y.astype(np.float32),pred)
            print("EPOCH {}  AUC Score {}".format(epoch,auc_score))
            schedular.step(auc_score)
            es(auc_score, model, model_path=f"model_fold_{fold}.bin")
            if es.early_stop:
                print("Early stopping")
                break        
            #if auc_score>best_score:
                        #best_score = auc_score 
                        #torch.save(model,'best_model.pth')
                        #print("Validation Score Improved ======>>>>>> Saving Model")
            del train_data,valid_data,label,valid_label
            gc.collect()

In [ ]:
train(0)
train(1)
train(2)
train(3)
train(4)

# **Prediction**

In [10]:
def predict(fold):
    test_df = pd.read_csv('/kaggle/input/jpeg-melanoma-384x384/test.csv')   #/kaggle/input/siim-isic-melanoma-classification/test.csv
    im_path = '/kaggle/input/jpeg-melanoma-384x384/test/' #'/kaggle/input/siic-isic-224x224-images/test/'
    batch_t = 32
    #model_path = '../working/model_fold_'+str(fold)+'.bin'
    model_path = '/kaggle/input/tpu-model/model_fold_'+str(fold)+'.bin' 
    
    
    test_im = test_df.image_name.values.tolist()
    test_y = np.ones(len(test_im))
    test_dataset = Data_Loader(im_path,test_im,test_y,valid=False)
    test_dataset = DataLoader(test_dataset,batch_t,shuffle=False,num_workers=4)
    device = 'cuda'
    
    
    model = EffNet()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    preds = []
    batch = 0
    #for i in range(5):
    for test_data,test_label in test_dataset:
                test_data = test_data.to(device)
                batch +=1
                
                with torch.no_grad():
                    out = model(test_data)
                    preds.append(out.cpu())
                    if batch%50==0 : print('Batch  {}'.format(batch))

    pred=np.vstack((preds)).ravel()
    return pred
        

In [11]:
predict_1 = predict(0)
predict_2 = predict(1)
predict_3 = predict(2)
predict_4 = predict(3)
predict_5 = predict(4)

Loaded pretrained weights for efficientnet-b4
Batch  50
Batch  100
Batch  150
Batch  200
Batch  250
Batch  300
Loaded pretrained weights for efficientnet-b4
Batch  50
Batch  100
Batch  150
Batch  200
Batch  250
Batch  300
Loaded pretrained weights for efficientnet-b4
Batch  50
Batch  100
Batch  150
Batch  200
Batch  250
Batch  300
Loaded pretrained weights for efficientnet-b4
Batch  50
Batch  100
Batch  150
Batch  200
Batch  250
Batch  300


In [13]:
prediction = (predict_1+predict_2+predict_3+predict_4+predict_5)/5
submission = pd.read_csv("../input/jpeg-melanoma-384x384/sample_submission.csv")
submission.loc[:,'target'] = prediction
submission.to_csv('submission.csv',index=False)

In [ ]:
s1 = pd.read_csv('../working/submission.csv')  # ../input/new-submit/submit_bce.csv
s2 = pd.read_csv('../input/new-submit/submit_fl.csv')
s3 = pd.read_csv('../input/new-submit/submit_bce.csv')
target_res = s3.target.values
target_eff = s2.target.values
target_eff_1 = s1.target.values
result = (target_res + target_eff + target_eff_1)/3
#result_1 = (target_eff + target_eff_1)/2
submission = pd.read_csv("../input/jpeg-melanoma-384x384/sample_submission.csv")
submission.loc[:,'target'] = result
submission.to_csv('submit_1.csv',index=False)

In [20]:
sub = pd.read_csv('../working/submission.csv')
sub.head()

,image_name,target
0,ISIC_0052060,-6.696511
1,ISIC_0052349,-6.909279
2,ISIC_0058510,-7.553364
3,ISIC_0073313,-7.886381
4,ISIC_0073502,-4.584024


**submission file**  -- single model on effnet b4 using bceloss

**submit_1 file**  -- combined model trained on bceloss and focal loss

In [ ]:
reduced_loss = xm.mesh_reduce("loss_reduce", loss, reduce_fn)